In [1]:
import random
import numpy as np
import spacy
from get_object_path import get_object_path

nlp = spacy.load("en_core_web_md")

### func-verb-alignment dict

In [1]:
# func-verb-alignment dict
func_verb_alignment = {
    'setText': {'enter', 'input'},
    'click': {'click'},
    'waitForElementPresent': {'wait'},
    'getText': {'get'},
    'verifyMatch': {'check'}
}

# verb-func alignment dict
verb_func_alignment = {}
for func, verbs in func_verb_alignment.items():
    for verb in verbs:
        verb_func_alignment[verb] = func
# verb_func_alignment
func_verb_alignment

{'setText': {'enter', 'input'},
 'click': {'click'},
 'waitForElementPresent': {'wait'},
 'getText': {'get'},
 'verifyMatch': {'check'}}

### Reformat output of object_path

In [3]:
def obj_path(test_case_dict):
    '''(dict) -> str
    Get object_path

    Params:
        (dict) One test case dict from a parsed manual test case

    Returns:
        (str) Object path
    '''

    output = get_object_path(test_case_dict)

    if type(output) == 'set':
        path = random.sample(output, 1)
        return f'findTestObject(\'{path}\')'
    else:
        return f'findTestObject(\'{output}\')'

### Alignment

In [4]:
def align_func(test_case_dict):
    '''(dict) -> str
    Get aligned WebUI function

    Params:
        (dict) One test case dict from a parsed manual test case

    Returns:
        (str) Aligned WebUI function name
    '''

    action = test_case_dict['V']
    if action in verb_func_alignment:
        func = verb_func_alignment[action]
    else:
        max_sim = -np.inf
        for verb in verb_func_alignment:
            similarity = nlp(action).similarity(nlp(verb))
            if similarity > max_sim:
                max_sim = similarity
                func = verb_func_alignment[verb]
    return func


def align_one_line_to_script(test_case_dict):
    '''(dict) -> dict
    Generate aligned script dict

    Params:
        (dict) One test case dict from a parsed manual test case

    Returns:
        (dict) Generated aligned script dict
    '''

    # Get aligned WebUI function
    fun = align_func(test_case_dict)

    # Get arguments for the WebUI function
    args = []

    if fun == 'setText':
        arg1 = obj_path(test_case_dict)
        arg2 = f"'{test_case_dict['value']}'"
        args = [arg1, arg2]
    if fun == 'click':
        arg1 = obj_path(test_case_dict)
        args = [arg1]
    if fun == 'waitForElementPresent':
        arg1 = obj_path(test_case_dict)
        args = [arg1, str(test_case_dict['time']), 'FailureHandling.STOP_ON_FAILURE']
    if fun == 'getText':
        args = [obj_path(test_case_dict)]

    script_dict = {"action": fun, "entities": args}

    # Get variable
    if fun == 'getText':
        obj = args[-1].split('\'')[-2].split('_')[-1]
        script_dict['var_name'] = 'actual' + obj
        script_dict['var_type'] = 'def'

    return script_dict

### Generate scripts

In [5]:
def generate_script_dicts(test_case_dicts):
    '''(list) -> list
    Generate aligned list of script dicts

    Params:
        (list) parsed test case dicts for one manual test case

    Returns:
        (list) Generated aligned list of script dicts
    '''

    script_dicts = []
    for line_dict in test_case_dicts:
        if line_dict['V'] not in verb_func_alignment:
            continue
        script_dict = align_one_line_to_script(line_dict)
        script_dicts.append(script_dict)
    return script_dicts


def dicts_to_script(script_dicts):
    '''(list) -> str
    Generate aligned script from script dicts

    Params:
        (list) Generated script dicts for one manual test case

    Returns:
        (str) Generated aligned script
    '''
    script = []
    variables = {}
    for line_dict in script_dicts:
        line = f"WebUI.{line_dict['action']}({', '.join(line_dict['entities'])})"
        if line_dict['action'] == 'verifyMatch':
            actual = variables['getText']
            expected = actual.replace("actual", "expected")
            line = f"WebUI.{line_dict['action']}({actual}, {expected}, false)"

        if 'var_name' in line_dict:
            line = f"{line_dict['var_type']} {line_dict['var_name']} = {line}"
            variables[line_dict['action']] = line_dict['var_name']

        script.append(line)
    return '\n'.join(script)


def generate_script(test_case):
    '''(list) -> list
    Generate aligned script

    Params:
        (list) parsed test case dicts for one manual test case

    Returns:
        (list) Generated aligned script
    '''
    script_dicts = generate_script_dicts(test_case)
    script = dicts_to_script(script_dicts)
    return script

In [7]:
script_dicts = generate_script_dicts(test_cases[1])
script = dicts_to_script(script_dicts)

In [29]:
# print("{'action': 'waitForElementPresent',\n 'entities': [\n\t\"findTestObject('Page_Login/txt_EmailErrorMessage')\",\n\t'3',\n\t'FailureHandling.STOP_ON_FAILURE']}")
print("WebUI.waitForElementPresent(\nfindTestObject('Page_Login/txt_EmailErrorMessage'), \n3, \nFailureHandling.STOP_ON_FAILURE\n)")

WebUI.waitForElementPresent(
findTestObject('Page_Login/txt_EmailErrorMessage'), 
3, 
FailureHandling.STOP_ON_FAILURE
)


In [9]:
script_dicts

[{'action': 'setText',
  'entities': ["findTestObject('Page_Login/tbx_Email')", "'invalid@wrong'"]},
 {'action': 'setText',
  'entities': ["findTestObject('Page_Login/tbx_Password')",
   "'invalidpassword'"]},
 {'action': 'click', 'entities': ["findTestObject('Page_Login/btn_Login')"]},
 {'action': 'waitForElementPresent',
  'entities': ["findTestObject('Page_Login/txt_EmailErrorMessage')",
   '3',
   'FailureHandling.STOP_ON_FAILURE']},
 {'action': 'getText',
  'entities': ["findTestObject('Page_Login/txt_EmailErrorMessage')"],
  'var_name': 'actualEmailErrorMessage',
  'var_type': 'def'},
 {'action': 'verifyMatch', 'entities': []},
 {'action': 'waitForElementPresent',
  'entities': ["findTestObject('Page_Login/txt_PasswordErrorMessage')",
   '3',
   'FailureHandling.STOP_ON_FAILURE']},
 {'action': 'getText',
  'entities': ["findTestObject('Page_Login/txt_PasswordErrorMessage')"],
  'var_name': 'actualPasswordErrorMessage',
  'var_type': 'def'},
 {'action': 'verifyMatch', 'entities': 

### Test with some parsed cases

In [6]:
test_cases = [
     [{'V': 'enter',
       'location': 'to email textbox',
       'value': 'admin1@mail.com',
       'page': 'login'},
      {'V': 'enter',
       'location': 'to password textbox',
       'value': 'Admin@123',
       'page': 'login'},
      {'V': 'click', 'value': 'Login', 'page': 'login'},
      {'V': 'wait',
       'value': 'title to be present for seconds',
       'time': 30,
       'page': 'login'}],
     [{'V': 'enter',
       'location': 'to email textbox',
       'value': 'invalid@wrong',
       'page': 'login'},
      {'V': 'enter',
       'location': 'to password textbox',
       'value': 'invalidpassword',
       'page': 'login'},
      {'V': 'click', 'value': 'Login', 'page': 'login'},
      {'V': 'wait',
       'value': 'email error message to appear for seconds',
       'time': 3,
       'page': 'login'},
      {'V': 'get', 'value': 'email error message', 'page': 'login'},
      {'V': 'check', 'page': 'login'},
      {'V': 'wait',
       'value': 'password error message to appear for seconds',
       'time': 3,
       'page': 'login'},
      {'V': 'get', 'value': 'password error message', 'page': 'login'},
      {'V': 'check', 'page': 'login'}],
     [{'V': 'click', 'value': 'the avatar', 'page': 'logout'},
      {'V': 'click', 'value': 'Sign out', 'page': 'logout'},
      {'V': 'wait', 'value': 'Login', 'time': 10, 'page': 'logout'}],
     [{'V': 'click', 'value': "user 's avatar", 'page': 'change password'},
      {'V': 'click', 'value': 'change password link', 'page': 'change password'},
      {'V': 'wait',
       'value': 'title to appear for s',
       'time': 10,
       'page': 'change password'},
      {'V': 'enter',
       'location': 'current password text box',
       'value': 'Admin@123',
       'page': 'change password'},
      {'V': 'enter',
       'location': 'new password text box',
       'value': 'Admin@1234',
       'page': 'change password'},
      {'V': 'enter',
       'location': 'to confirm password',
       'value': 'Admin@1234',
       'page': 'change password'},
      {'V': 'click', 'value': 'submit', 'page': 'change password'}],
     [{'V': 'click', 'value': 'the avatar', 'page': 'change password'},
      {'V': 'click', 'value': 'Change password', 'page': 'change password'},
      {'V': 'wait',
       'value': 'title to appear for s',
       'time': 10,
       'page': 'change password'},
      {'V': 'enter',
       'location': 'current password text box',
       'value': ' ',
       'page': 'change password'},
      {'V': 'enter',
       'location': 'new password text box',
       'value': ' ',
       'page': 'change password'},
      {'V': 'enter',
       'location': 'to confirm password',
       'value': ' ',
       'page': 'change password'},
      {'V': 'click', 'value': 'submit', 'page': 'change password'}],
     [{'V': 'click',
       'value': 'my profile link',
       'page': 'Update my personal info'},
      {'V': 'wait',
       'value': 'to appear for s',
       'time': 20,
       'page': 'Update my personal info'},
      {'V': 'enter',
       'location': 'full name text box',
       'value': 'new name',
       'page': 'Update my personal info'},
      {'V': 'enter',
       'location': 'phone number text box',
       'value': '1234567890',
       'page': 'Update my personal info'},
      {'V': 'click', 'value': 'submit', 'page': 'Update my personal info'}],
     [{'V': 'click', 'value': 'the avatar', 'page': 'Update my personal info'},
      {'V': 'click', 'value': 'My Profile', 'page': 'Update my personal info'},
      {'V': 'wait',
       'value': 'for email to appear for s',
       'time': 20,
       'page': 'Update my personal info'},
      {'V': 'enter',
       'location': 'full name text box',
       'value': ' ',
       'page': 'Update my personal info'},
      {'V': 'enter',
       'location': 'phone number text box',
       'value': '12345678',
       'page': 'Update my personal info'},
      {'V': 'click', 'value': 'submit', 'page': 'Update my personal info'}],

     [{'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'click', 'value': 'create library button', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'input',
       'location': 'to the name field textbox',
       'value': 'new library',
       'page': 'Create Libraries'},
      {'V': 'click', 'value': 'submit button', 'page': 'Create Libraries'}],
     [{'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'click', 'value': 'Create Library', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'set', 'page': 'Create Libraries'},
      {'V': 'click', 'value': 'Submit', 'page': 'Create Libraries'},
      {'V': 'wait',
       'value': 'for invalid message text to be present for s',
       'time': 20,
       'page': 'Create Libraries'},
      {'V': 'get', 'value': 'invalid message text', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'}],
     [{'V': 'wait',
       'value': 'Add Library',
       'time': 20,
       'page': 'Create Libraries'},
      {'V': 'click', 'value': 'Add Library', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'set', 'page': 'Create Libraries'},
      {'V': 'click', 'value': 'Submit', 'page': 'Create Libraries'},
      {'V': 'wait',
       'value': 'for success message text to be present for s',
       'time': 20,
       'page': 'Create Libraries'},
      {'V': 'get', 'value': 'success message text', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'}],
     [{'V': 'click', 'value': 'Add Library', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'click', 'value': 'Submit', 'page': 'Create Libraries'},
      {'V': 'wait',
       'value': 'for invalid message text to be present for s',
       'time': 20,
       'page': 'Create Libraries'},
      {'V': 'get', 'value': 'invalid message text', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'click', 'value': 'Cancel', 'page': 'Create Libraries'}],
     [{'V': 'click',
       'value': 'library navigation item',
       'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'verify', 'page': 'Create Libraries'},
      {'V': 'click',
       'value': 'library navigation item',
       'page': 'Create Libraries'}],
     [{'V': 'click',
       'value': 'library name with rely on name',
       'page': 'Update Libraries'},
      {'V': 'click',
       'value': 'update option with rely on name',
       'page': 'Update Libraries'},
      {'V': 'verify', 'page': 'Update Libraries'},
      {'V': 'click', 'value': 'submit button', 'page': 'Update Libraries'}]
]

In [7]:
scripts = [generate_script(test_case) for test_case in test_cases]

for i, script in enumerate(scripts):
    print("Script " + str(i)+'\n')
    print(script)
    print("-"*100)

Script 0

WebUI.setText(findTestObject('Page_Login/tbx_Email'), 'admin1@mail.com')
WebUI.setText(findTestObject('Page_Login/tbx_Password'), 'Admin@123')
WebUI.click(findTestObject('Page_Login/btn_Login'))
WebUI.waitForElementPresent(findTestObject('Page_ChangePassword/txt_Title'), 30, FailureHandling.STOP_ON_FAILURE)
----------------------------------------------------------------------------------------------------
Script 1

WebUI.setText(findTestObject('Page_Login/tbx_Email'), 'invalid@wrong')
WebUI.setText(findTestObject('Page_Login/tbx_Password'), 'invalidpassword')
WebUI.click(findTestObject('Page_Login/btn_Login'))
WebUI.waitForElementPresent(findTestObject('Page_Login/txt_EmailErrorMessage'), 3, FailureHandling.STOP_ON_FAILURE)
def actualEmailErrorMessage = WebUI.getText(findTestObject('Page_Login/txt_EmailErrorMessage'))
WebUI.verifyMatch(actualEmailErrorMessage, expectedEmailErrorMessage, false)
WebUI.waitForElementPresent(findTestObject('Page_Login/txt_PasswordErrorMessage'),